In [1]:
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Add, Flatten, Normalization, Input, Resizing, Conv2D, MaxPooling2D

from tensorflow.keras.regularizers import l2
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt

from src.const import AUDIO_PATH, MAIN_LABELS, BATCH_SIZE, VALIDATION_SPLIT, SEED

from src.preprocess import load_and_preprocess, load_augmented_data

In [2]:
# train_audio_with_augmented, train_labels_with_augmented = load_augmented_data()

### Unknown vs known task

In [ ]:
train_ds_bin_X, train_ds_bin_y, val_ds_bin_X, val_ds_bin_y, train_ds_main_X, train_ds_main_y, val_ds_main_X, val_ds_main_y = load_and_preprocess(plot_samples = True)

In [5]:
model = Sequential()
model.add(Bidirectional(LSTM(128), merge_mode="sum"))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation="sigmoid", kernel_regularizer=l2(0.01)))

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [30]:
input_shape=(63,128)
model = tf.keras.Sequential()
model.add(LSTM(128))
model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
    loss="binary_crossentropy",
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [38]:
val_ds_bin_X[val_ds_bin_X == -np.inf] = np.min(train_ds_bin_X)

In [ ]:
history = model.fit(
    x=train_ds_bin_X, 
    y=np.expand_dims(train_ds_bin_y, axis=1), 
    batch_size=1024, 
    epochs=50, 
    validation_data=(val_ds_bin_X, np.expand_dims(val_ds_bin_y, axis=1)), 
    callbacks=[es]
)